# Laboratorio 3

In [50]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('sql-murder-mystery.db')
cursorObj = conn.cursor()


## Contexto 
Se ha cometido un delito y la policía necesita su ayuda. Un detective te dio el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un asesinato que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL City. Empiece por recuperar el informe de la escena del crimen correspondiente de la base de datos del departamento de policía.

In [51]:
tables = pd.read_sql("SELECT * FROM sqlite_master", conn)
tables.head()

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,CREATE TABLE crime_scene_report (\n dat...
1,table,drivers_license,drivers_license,3,CREATE TABLE drivers_license (\n id int...
2,table,person,person,4,CREATE TABLE person (\n id integer PRIM...
3,table,facebook_event_checkin,facebook_event_checkin,5,CREATE TABLE facebook_event_checkin (\n ...
4,table,interview,interview,6,CREATE TABLE interview (\n person_id in...


In [52]:
crime_table = pd.read_sql("SELECT * FROM crime_scene_report;", conn)
crime_table.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


In [53]:
crime_scene = pd.read_sql("""SELECT description 
FROM 
crime_scene_report 
WHERE date = 20180115 AND
city = 'SQL City' and type = 'murder'""", conn )
print(crime_scene)

                                         description
0  Security footage shows that there were 2 witne...


# Testigos

In [54]:
testigo_1 = pd.read_sql("""select id, name 
from person 
where address_street_name = 'Northwestern Dr' 
order by address_number DESC limit 1""", conn)
print(testigo_1)

      id            name
0  14887  Morty Schapiro


In [55]:
testigo_2 = pd.read_sql("select id, name from person where address_street_name = 'Franklin Ave' and name like 'Annabel%'", conn)
print(testigo_2)

      id            name
0  16371  Annabel Miller


In [56]:
transcript = pd.read_sql("SELECT * FROM interview WHERE person_id=16371 OR person_id=14887", conn)
print(transcript)

   person_id                                         transcript
0      14887  I heard a gunshot and then saw a man run out. ...
1      16371  I saw the murder happen, and I recognized the ...


# Culpable 

In [57]:
pista_1 = pd.read_sql("select person.name from get_fit_now_member join person on get_fit_now_member.person_id = person.id join drivers_license on person.license_id = drivers_license.id where membership_status = 'gold' and get_fit_now_member.id like '48Z%' and plate_number like '%H42W%'", conn)
print(pista_1)

            name
0  Jeremy Bowers


In [58]:
entrevista = pd.read_sql("select transcript from interview join person on person.id = interview.person_id where name = 'Jeremy Bowers'", conn)
print(entrevista)

                                          transcript
0  I was hired by a woman with a lot of money. I ...


# Mente maestra

In [59]:
asesino_2 = pd.read_sql("select name from person join facebook_event_checkin on person.id = facebook_event_checkin.person_id join drivers_license on person.license_id = drivers_license.id where event_name = 'SQL Symphony Concert' and date like '201712%' and car_make = 'Tesla'and car_model = 'Model S' and height between 65 and 67 and hair_color = 'red' and gender='female'group by person.id having count(*) == 3 ", conn)
print(asesino_2)

               name
0  Miranda Priestly


In [60]:
cursor = conn.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')" 
cursor.execute(sql)
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [61]:
cursor = conn.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')" 
cursor.execute(sql)
conn.commit()
var = pd.read_sql("SELECT value FROM solution", conn)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'